In [5]:
import oracledb
import pandas as pd
from flask import Flask,jsonify,request
import os
import datetime as dt
from datetime import timedelta

username=os.getlogin()
userdb="ioliva"
dsndb="192.156.141.120:1521/comsp.igt.com"
date_formatexport = "%d-%m-%Y"
date_format = "%d.%m.%Y"
hoy = dt.datetime.now()
rango = hoy - timedelta(days=30)
fecha_actual = hoy.strftime(date_formatexport)
fecha_mesant = rango.strftime(date_formatexport)
folder_path = f'C:\\Users\\{username}\\Documents\\Py\\ARCHIVOS PROCESO AM\\'
dir_b2b = folder_path + f'GENERADOS\\B2B\\B2B CIERRES {fecha_actual}.csv'
userpwd = input()

oracledb.init_oracle_client(lib_dir=r"C:\instantclient_23_6")
def conectar(userdb,userpwd,dsndb):
    conn = oracledb.connect(user=userdb, password=userpwd, dsn=dsndb)
    return conn

conn = conectar(userdb,userpwd,dsndb)
cursor = conn.cursor()


In [27]:
conn.is_healthy()
conn.close()

In [ ]:
df_agv = pd.DataFrame(columns=['AGENCIA_ID','NUMERO_AGENCIA','ESTADO_VIGENCIA','LIMITE_CREDITO','CANAL'])
canal = "B2B"
vigencia = "V"
sql = "SELECT AGENCIA_ID, NUMERO_AGENCIA, ESTADO_VIGENCIA, LIMITE_CREDITO, CANAL from GTECH.AGENCIAS WHERE CANAL = :vcan AND ESTADO_VIGENCIA = :vvig"
for row in cursor.execute(sql,vcan=canal,vvig=vigencia):
    df_agv.loc[len(df_agv)] = row
    print(row)

In [ ]:
app = Flask(__name__)
canal = "B2B"
vigencia = "V"
@app.route('/b2b/vigentes',methods=['GET'])    
def obtener_datos():
    df_agv = pd.DataFrame(columns=['AGENCIA_ID','NUMERO_AGENCIA','ESTADO_VIGENCIA','LIMITE_CREDITO','CANAL'])
    sql = "SELECT AGENCIA_ID, NUMERO_AGENCIA, ESTADO_VIGENCIA, LIMITE_CREDITO, CANAL from GTECH.AGENCIAS WHERE CANAL = :vcan AND ESTADO_VIGENCIA = :vvig"
    for row in cursor.execute(sql,vcan=canal,vvig=vigencia):
        df_agv.loc[len(df_agv)] = row
    print(row)
    return jsonify(df_agv.to_dict(orient='records'))
if __name__ == '__main__':
    app.run(debug=False)

In [ ]:
agencia_id = 18045
dfq_agencias = pd.DataFrame(columns=['DEPOSITO_ID','AGENCIA_ID','NUM_CIERRE_DEPOSITO','FECHA_CIERRE_DEPOSIT','ESTADO'])
sql = "SELECT DEPOSITO_ID, AGENCIA_ID, NUM_CIERRE_DEPOSITO, FECHA_CIERRE_DEPOSIT, ESTADO FROM GTECH.DEPOSITOS WHERE AGENCIA_ID = :vaid"
for row in cursor.execute(
    sql,vaid=str(agencia_id)
    ):
    dfq_agencias.loc[len(dfq_agencias)] = row
    print(row)

In [ ]:
montodep = input()
dfq_buscaconc = pd.DataFrame(columns=['AGENCIA','CIERRE','MONTO','FECHA_CIERRE','HORA_CONC', 'EST', 'EJECUTIVO'])
sql = """SELECT DISTINCT a.NUMERO_AGENCIA, d.NUM_CIERRE_DEPOSITO, d.TOTAL_DEPOSITO, d.FECHA_CIERRE_DEPOSIT, dd.FECHA_CONC_EF,d.ESTADO_PROCESO, e.NOMBRE 
        FROM AGENCIAS a 
        INNER JOIN .AGENTES ag ON a.AGENTE_ID = ag.AGENTE_ID 
        INNER JOIN .DEPOSITOS d ON a.AGENCIA_ID = d.AGENCIA_ID 
        INNER JOIN .DETALLE_DEPOSITOS dd ON d.DEPOSITO_ID = dd.DEPOSITO_ID 
        INNER JOIN .EJECUTIVOS e ON e.EJECUTIVO_ID = a.EJECUTIVO_ID 
        WHERE d.TOTAL_DEPOSITO = :vmontodep"""
for row in cursor.execute(sql,vmontodep=montodep):
    dfq_buscaconc.loc[len(dfq_buscaconc)] = row
print(dfq_buscaconc)


In [47]:
b2bbechids = [16640,18045,14603]
#def format_list(ids):
#    return '' + ','.join(f"{n}" for n in ids)+''
#b2bbech=format_list(b2bbechids)
#eproc= 'DP'
dfq_b2bbech = pd.DataFrame(columns=['AGENCIA','CIERRE','MONTO','FECHA_CIERRE', 'EST'])
sql = """SELECT a.NUMERO_AGENCIA, d.NUM_CIERRE_DEPOSITO, d.TOTAL_DEPOSITO, d.FECHA_CIERRE_DEPOSIT, d.ESTADO_PROCESO 
        FROM GTECH.DEPOSITOS d 
        INNER JOIN GTECH.AGENCIAS a ON d.AGENCIA_ID = a.AGENCIA_ID
        WHERE a.AGENCIA_ID IN (:a0,:a1,:a2)
        AND d.ESTADO_PROCESO = 'DP'"""
for row in cursor.execute(sql,a0=b2bbechids[0],a1=b2bbechids[1],a2=b2bbechids[2]):
    dfq_b2bbech.loc[len(dfq_b2bbech)] = row
dfq_b2bbech.sort_values(by=['FECHA_CIERRE','AGENCIA'], ascending=[True, True], inplace=True)

In [ ]:
dfq_b2bbech.to_csv(dir_b2b, sep=';', index=False, header=False, float_format='%.0f', date_format=date_formatexport)

In [ ]:
nagencia = input()

dfq_hconc = pd.DataFrame(columns=['AGENCIA','CIERRE','MONTO','FECHA_CIERRE','FECHA_CONC_EF', 'EST'])
sql = """SELECT DISTINCT a.NUMERO_AGENCIA,d.NUM_CIERRE_DEPOSITO,d.TOTAL_DEPOSITO, d.FECHA_CIERRE_DEPOSIT,dd.FECHA_CONC_EF ,d.ESTADO_PROCESO FROM GTECH.DEPOSITOS d
        INNER JOIN GTECH.AGENCIAS a ON d.AGENCIA_ID = a.AGENCIA_ID
        INNER JOIN GTECH.EJECUTIVOS e ON a.EJECUTIVO_ID = e.EJECUTIVO_ID
        INNER JOIN GTECH.DETALLE_DEPOSITOS dd ON dd.DEPOSITO_ID = d.DEPOSITO_ID 
        WHERE a.NUMERO_AGENCIA = :vnagencia
        AND d.FECHA_CIERRE_DEPOSIT >= TO_DATE (:vfecha,'dd-mm-yyyy')
        """

if conn.is_healthy():
    for row in cursor.execute(sql,vnagencia=nagencia,vfecha=fecha_mesant):
        dfq_hconc.loc[len(dfq_hconc)] = row
else:
   print(f"Generar instancia conección")

dfq_hconc.sort_values(by=['FECHA_CIERRE'], ascending=True, inplace=True)

In [26]:
dfq_mc = pd.DataFrame(columns=['MONTO','AGENCIA'])

sql = """SELECT DISTINCT d.TOTAL_DEPOSITO,a.NUMERO_AGENCIA FROM GTECH.DEPOSITOS d
        INNER JOIN GTECH.AGENCIAS a ON d.AGENCIA_ID = a.AGENCIA_ID
        WHERE d.FECHA_CIERRE_DEPOSIT >= TO_DATE ('01-01-2024','dd-mm-yyyy') AND a.CANAL = 'RETAIL' AND d.TOTAL_DEPOSITO != 0"""
for row in cursor.execute(sql):
    dfq_mc.loc[len(dfq_mc)] = row
dfq_mc.sort_values(by=['MONTO','AGENCIA'], ascending=[True, True], inplace=True)

#dfq_mc.to_csv(dir_b2b, sep=';', index=False, header=True, float_format='%.0f')